## 超參數調整
- GridSearch
- RandomizedSearch
- BasianSearch
    - better, 調整參數過程也是一個最佳化過程!
    - [kerastuner](https://keras.io/keras_tuner/)

In [4]:
!pip install keras-tuner

     |████████████████████████████████| 96 kB 2.5 MB/s 


In [5]:
import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras import layers
import kerastuner as kt
import sklearn

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  after removing the cwd from sys.path.


In [10]:
from sklearn.datasets import load_boston
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

In [12]:
# sklearn 來展示GridSearch(keras-tuner也有) 

X, y = load_boston(return_X_y=True)
housing_rf = RandomForestRegressor()

X.shape, y.shape

((506, 13), (506,))

In [27]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, y)

In [28]:
x_train.shape, y_train.shape

((379, 13), (379,))

In [13]:
X[0]

array([6.320e-03, 1.800e+01, 2.310e+00, 0.000e+00, 5.380e-01, 6.575e+00,
       6.520e+01, 4.090e+00, 1.000e+00, 2.960e+02, 1.530e+01, 3.969e+02,
       4.980e+00])

In [14]:
y[0]

24.0

In [11]:
param_grid = {
    'max_depth': [5, 10, 100],
    'n_estimators': [100, 150, 200]
}

In [15]:
grid_search_rf = GridSearchCV(
    estimator=housing_rf,
    param_grid=param_grid,
)
grid_search_rf.fit(X, y)

GridSearchCV(cv=None, error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mse', max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators=100, n_jobs=None,
                                             oob_score=False, random_state=None,
                                             verbose=0, warm_start=False),
             iid='deprecated', n

In [16]:
# 透過attritube 取得 最佳超參數

grid_search_rf.best_params_

{'max_depth': 10, 'n_estimators': 200}

In [17]:
from kerastuner import HyperParameters as hp

In [21]:
# 試試keras-tuner

def build_model(hp):
    model = keras.Sequential([
        layers.Dense(
            hp.Int(name='first_hidden', min_value=16, max_value=64, step=16),
            activation='relu'
        ),
        layers.Dense(
            hp.Int(name='second_hidden', min_value=16, max_value=64, step=16),
            activation='relu'
        ),
        layers.Dense(
            units=1
        )
    ])
    
    model.compile(
        optimizer=keras.optimizers.Adam(
            learning_rate=hp.Float('lr', min_value=0.005, max_value=0.01, sampling='log')
        ),
        loss=keras.losses.MeanSquaredError(),
        metrics=['mse']
    )
    return model

In [22]:
tuner = kt.BayesianOptimization(
    hypermodel=build_model,
    objective='val_mse',
    max_trials=10
)

In [30]:
tuner.search_space_summary()

Search space summary
Default search space size: 3
first_hidden (Int)
{'default': None, 'conditions': [], 'min_value': 16, 'max_value': 64, 'step': 16, 'sampling': None}
second_hidden (Int)
{'default': None, 'conditions': [], 'min_value': 16, 'max_value': 64, 'step': 16, 'sampling': None}
lr (Float)
{'default': 0.005, 'conditions': [], 'min_value': 0.005, 'max_value': 0.01, 'step': None, 'sampling': 'log'}


In [31]:
tuner.search(x_train, y_train.reshape(-1,  1), epochs=10, validation_data=(x_test, y_test.reshape(-1, 1)))

Trial 10 Complete [00h 00m 01s]
val_mse: 60.93595504760742

Best val_mse So Far: 56.03102111816406
Total elapsed time: 00h 00m 20s
INFO:tensorflow:Oracle triggered exit


In [32]:
# 取得最好的模型(可以多個)
models = tuner.get_best_models(num_models=2)

In [33]:
# 可以多個

tuner.results_summary()

Results summary
Results in ./untitled_project
Showing 10 best trials
Objective(name='val_mse', direction='min')
Trial summary
Hyperparameters:
first_hidden: 48
second_hidden: 48
lr: 0.008256458795369386
Score: 56.03102111816406
Trial summary
Hyperparameters:
first_hidden: 32
second_hidden: 64
lr: 0.01
Score: 56.861473083496094
Trial summary
Hyperparameters:
first_hidden: 32
second_hidden: 16
lr: 0.01
Score: 58.30662155151367
Trial summary
Hyperparameters:
first_hidden: 64
second_hidden: 16
lr: 0.005
Score: 60.44956970214844
Trial summary
Hyperparameters:
first_hidden: 48
second_hidden: 64
lr: 0.009098553304137294
Score: 60.93595504760742
Trial summary
Hyperparameters:
first_hidden: 32
second_hidden: 48
lr: 0.01
Score: 61.14206314086914
Trial summary
Hyperparameters:
first_hidden: 64
second_hidden: 48
lr: 0.008837235278299311
Score: 61.21369934082031
Trial summary
Hyperparameters:
first_hidden: 64
second_hidden: 64
lr: 0.006304413524087287
Score: 61.46294021606445
Trial summary
Hyperpar